model

In [1]:
import keras
#from cnnmodel import model
from keras.preprocessing.image import ImageDataGenerator, load_img
import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf
import os, random, cv2
from keras.utils import to_categorical

/opt/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def createFileList(myDir):
    format = ('.tiff','.jpg','.png','.PNG')
    fileList = []
    print(os.walk(myDir, topdown=False))
    for root, dirs, files in os.walk(myDir, topdown=False):
        
        for name in files:
            if name.endswith(format):
                fullName = os.path.join(root, name)
                fileList.append(fullName)
    return fileList 

In [3]:
classes={0:'angry', 1:'fear', 2:'happy', 3:'neutral', 4:'sad', 5:'surprise'}




batch_size = 64
epochs = 60

train_x = [] #(10000,1,227,227) (1,227,227)에 대해 append
train_y = [] #(10000, 6) 


#train_dir에 적용
#e.g. case sad
#path 읽어 = [(1,227,227)만들기 |  파싱하여 label 얻기] 

#train_x.append(q)
'''
z = load_img(path = './data/train/sad/KA.SA2.34.tiff', grayscale=True, target_size=(227,227),
    interpolation='nearest')
q=np.asarray(z).astype('float32')
print(np.asarray([q]).shape)
q = np.asarray([q])
'''

fileList = []
fileList = createFileList('./data/train')



for x in fileList:
    z = load_img(path =x, grayscale=True, target_size=(227,227),
    interpolation='nearest')
    q=np.asarray(z).astype('float32')
    q = np.asarray([q])
    train_x.append(q)
    
    t = -1
    for y in classes.items():
        if y[1] in x:
            t = y[0]
        
    train_y.append(to_categorical(t, 6))

train_x = np.asarray(train_x)
train_y = np.asarray(train_y)
print(train_x.shape)   
print(train_y.shape)   

train_len = train_x.shape[0]

<generator object walk at 0x7f3ce9187f10>
(1793, 1, 227, 227)
(1793, 6)


In [4]:
#이상  train_x/ train_y 구성 끝

#val_x/ val_y 도 일단 똑같이 구성

val_x = []
val_y = []


fileList = createFileList('./data/valid')



for x in fileList:
    z = load_img(path =x, grayscale=True, target_size=(227,227),
    interpolation='nearest')
    q=np.asarray(z).astype('float32')
    q = np.asarray([q])
    val_x.append(q)
    
    t = -1
    for y in classes.items():
        if y[1] in x:
            t = y[0]
        
    val_y.append(to_categorical(t, 6))

val_x = np.asarray(val_x)
val_y = np.asarray(val_y)
print(val_x.shape)   
print(val_y.shape)   

val_len = train_x.shape[0]


<generator object walk at 0x7f3ce912df68>
(199, 1, 227, 227)
(199, 6)


In [5]:
train_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    #rescale=1./255,
    rotation_range=10,
    shear_range=0.1,
    fill_mode = 'constant',
    cval = 0,
    horizontal_flip=True)

print(train_x.shape)
train_x_moveaxis = np.moveaxis(train_x, 1, 3)  
print(train_x_moveaxis.shape)
train_datagen.fit(train_x_moveaxis)

train_x_mean = train_datagen.mean

(1793, 1, 227, 227)
(1793, 227, 227, 1)


In [6]:
print(val_x.shape)

(199, 1, 227, 227)


In [7]:
val_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    #rescale=1./255, 
)
val_x_moveaxis = np.moveaxis(val_x, 1, 3)  
val_datagen.fit(val_x_moveaxis)
#val_datagen.mean = x꺼 배끼끼
print(val_datagen.mean)
val_datagen.mean = train_x_mean
print(val_datagen.mean)

[[[121.65533]]]
[[[118.49298]]]


In [8]:
#sequential model definition
#model.compile
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adadelta
from keras.layers.normalization import BatchNormalization


def model_generate():
    img_rows, img_cols = 227, 227
    model = Sequential()
    model.add(Convolution2D(36, 11, 11, border_mode='valid',
                            input_shape=(img_rows, img_cols,1)))
    model.add(BatchNormalization())
    model.add(keras.layers.advanced_activations.PReLU(init='zero', weights=None))
    model.add(keras.layers.convolutional.ZeroPadding2D(padding=(2, 2), dim_ordering='tf'))
    model.add(MaxPooling2D(pool_size=(5, 5),strides=(2, 2)))
      
    model.add(keras.layers.convolutional.ZeroPadding2D(padding=(1, 1), dim_ordering='tf')) 
    model.add(Convolution2D(64, 3, 3))
    model.add(BatchNormalization())
    model.add(keras.layers.advanced_activations.PReLU(init='zero', weights=None))
    model.add(keras.layers.convolutional.ZeroPadding2D(padding=(1, 1), dim_ordering='tf')) 
    
    model.add(Convolution2D(64, 3, 3))
    model.add(BatchNormalization())
    model.add(keras.layers.advanced_activations.PReLU(init='zero', weights=None))
    model.add(keras.layers.convolutional.AveragePooling2D(pool_size=(3, 3),strides=(2, 2)))
     
    model.add(keras.layers.convolutional.ZeroPadding2D(padding=(1, 1), dim_ordering='tf'))
    model.add(Convolution2D(128, 3, 3))
    model.add(BatchNormalization())
    model.add(keras.layers.advanced_activations.PReLU(init='zero', weights=None))
    
    model.add(keras.layers.convolutional.ZeroPadding2D(padding=(1, 1), dim_ordering='tf'))
    model.add(Convolution2D(128, 3, 3))
    model.add(BatchNormalization())
    model.add(keras.layers.advanced_activations.PReLU(init='zero', weights=None))
     
    model.add(keras.layers.convolutional.ZeroPadding2D(padding=(1, 1), dim_ordering='tf'))
    model.add(keras.layers.convolutional.AveragePooling2D(pool_size=(3, 3),strides=(2, 2)))
     
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(BatchNormalization())
    model.add(keras.layers.advanced_activations.PReLU(init='zero', weights=None))
    model.add(Dropout(0.2))
    
    model.add(Dense(1024))
    model.add(BatchNormalization())
    model.add(keras.layers.advanced_activations.PReLU(init='zero', weights=None))
    model.add(Dropout(0.2))
     
      
    model.add(Dense(6))
    model.add(BatchNormalization())
      
    model.add(Activation('softmax'))

    ada = Adadelta(lr=0.1, rho=0.95, epsilon=1e-08)
    #model = multi_gpu_model(model, gpus=2)
    model.compile(loss='categorical_crossentropy',
                  optimizer=ada,
                  metrics=['accuracy'])
    model.summary()
    return model

model = model_generate()


filepath='model5_total_#4set_batch50_interpol_const.hdf5'
checkpointer = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, mode='auto')



/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (11, 11), input_shape=(227, 227,..., padding="valid")`
  
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `PReLU` call to the Keras 2 API: `PReLU(weights=None, alpha_initializer="zero")`
  app.launch_new_instance()
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `ZeroPadding2D` call to the Keras 2 API: `ZeroPadding2D(padding=(2, 2), data_format="channels_last")`
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Update your `ZeroPadding2D` call to the Keras 2 API: `ZeroPadding2D(padding=(1, 1), data_format="channels_last")`
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launc

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 217, 217, 36)      4392      
_________________________________________________________________
batch_normalization_1 (Batch (None, 217, 217, 36)      144       
_________________________________________________________________
p_re_lu_1 (PReLU)            (None, 217, 217, 36)      1695204   
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 221, 221, 36)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 109, 109, 36)      0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 111, 111, 36)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 109, 109, 64)      20800     
__________

/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: UserWarning: Update your `PReLU` call to the Keras 2 API: `PReLU(weights=None, alpha_initializer="zero")`
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: UserWarning: Update your `ZeroPadding2D` call to the Keras 2 API: `ZeroPadding2D(padding=(1, 1), data_format="channels_last")`
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: UserWarning: Update your `PReLU` call to the Keras 2 API: `PReLU(weights=None, alpha_initializer="zero")`
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:52: UserWarning: Update your `PReLU` call to the Keras 2 API: `PReLU(weights=None, alpha_initializer="zero")`


In [9]:
model.fit_generator(train_datagen.flow(train_x_moveaxis, train_y, batch_size =batch_size),
                    #steps_per_epoch = train_len/batch_size*12,
                    steps_per_epoch = train_len/batch_size*4,
                    epochs = epochs,
                    validation_data = val_datagen.flow(val_x_moveaxis,val_y,batch_size = batch_size),
                    callbacks = [checkpointer])

#l = train_datagen.flow(train_x_moveaxis, train_y, batch_size =batch_size)

#next(l)[0].shape

Epoch 1/60
113/112 [==============================] - 80s 709ms/step - loss: 0.9974 - acc: 0.7136 - val_loss: 0.7637 - val_acc: 0.8090

Epoch 00001: saving model to model5_total_#4set_batch50_interpol_const.hdf5
Epoch 2/60
113/112 [==============================] - 41s 365ms/step - loss: 0.7190 - acc: 0.8636 - val_loss: 0.6800 - val_acc: 0.8593

Epoch 00002: saving model to model5_total_#4set_batch50_interpol_const.hdf5
Epoch 3/60
113/112 [==============================] - 41s 361ms/step - loss: 0.6416 - acc: 0.8919 - val_loss: 0.6712 - val_acc: 0.8693

Epoch 00003: saving model to model5_total_#4set_batch50_interpol_const.hdf5
Epoch 4/60
113/112 [==============================] - 41s 365ms/step - loss: 0.5772 - acc: 0.9475 - val_loss: 0.6791 - val_acc: 0.8844

Epoch 00004: saving model to model5_total_#4set_batch50_interpol_const.hdf5
Epoch 5/60
113/112 [==============================] - 42s 367ms/step - loss: 0.5264 - acc: 0.9575 - val_loss: 0.6369 - val_acc: 0.8794

Epoch 00005: sav


Epoch 00039: saving model to model5_total_#4set_batch50_interpol_const.hdf5
Epoch 40/60
113/112 [==============================] - 41s 366ms/step - loss: 0.2878 - acc: 0.9649 - val_loss: 0.3977 - val_acc: 0.9397

Epoch 00040: saving model to model5_total_#4set_batch50_interpol_const.hdf5
Epoch 41/60
113/112 [==============================] - 41s 366ms/step - loss: 0.2785 - acc: 0.9825 - val_loss: 0.4071 - val_acc: 0.9447

Epoch 00041: saving model to model5_total_#4set_batch50_interpol_const.hdf5
Epoch 42/60
113/112 [==============================] - 42s 368ms/step - loss: 0.2636 - acc: 0.9825 - val_loss: 0.3901 - val_acc: 0.9296

Epoch 00042: saving model to model5_total_#4set_batch50_interpol_const.hdf5
Epoch 43/60
113/112 [==============================] - 41s 365ms/step - loss: 0.2725 - acc: 0.9737 - val_loss: 0.3953 - val_acc: 0.9447

Epoch 00043: saving model to model5_total_#4set_batch50_interpol_const.hdf5
Epoch 44/60
113/112 [==============================] - 41s 366ms/step -

A total data # * steps for epochs / batchsize  ==  172 
Therefore, 172 * 32(batchsize) == A total data #

In [10]:
imgs, labels = next(train_data_generator)
print(labels.shape)
print(imgs[0].shape)
imgplot=plt.imshow(imgs[0][0], cmap='gray')
#plots(imgs, titles=labels)
#c=imgs[0][:,:,0]
#plt.imshow(c)
#plt.show()



NameError: name 'train_data_generator' is not defined

In [ ]:
#setting data at specific path = './data/train

In [ ]:
#load model

In [ ]:
#model fit_generator